# Anatomical Preprocessing

This notebooks preprocesses anatomical MRI images by executing the following processing steps:

1. Reorient images to RAS
1. Crop FOV with FSL
1. N4-inhomogenity correction with ANTS
1. GM, WM and CSF segmentation with SPM
1. Brainmask creation and brain extraction with Nilearn
1. Normalization to ICBM template with ANTS

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format:

    dataset
    ├── analysis-anat_specs.json
    └── sub-{sub_id}
        └── anat
            └── sub-{sub_id}*{T1w_id}*.nii.gz
            
**Note:** Subfolders for individual scan sessions are optional. `fmriflows` will run the preprocessing on all files of a subject.

## Execution Specifications

This notebook will extract the relevant processing specifications from the `fmriflows_spec_preproc.json` file in the dataset folder.

In [ ]:
import json
from os.path import join as opj

spec_file = opj('/data', 'fmriflows_spec_preproc.json')

with open(spec_file) as f:
    specs = json.load(f)

In [ ]:
# Extract parameters for anatomical preprocessing workflow
subject_list = specs['subject_list_anat']
session_list = specs['session_list_anat']
T1w_id = specs['T1w_id']
res_norm = specs['res_norm']
n_stages = specs['steps_of_norm']
n_proc = specs['n_parallel_jobs']

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject identifiers
subject_list

In [ ]:
# List of session identifiers
session_list

In [ ]:
# Anatomical image identifier
T1w_id

In [ ]:
# Resolution of normalized images
res_norm

In [ ]:
# 2 or 3 steps ANTs normalization
n_stages

In [ ]:
# Number of parallel jobs to run
n_proc

# Creating the Workflow

To ensure a good overview of the anatomical preprocessing, the workflow was divided into two subworkflows:

1. The Main Workflow, i.e. doing the actual preprocessing
2. Report Workflow, i.e. visualizating relevant steps for quality control

## Import Modules

In [ ]:
from os.path import join as opj
from nipype import Node, Workflow, Function, IdentityInterface
from nipype.interfaces.image import Reorient
from nipype.interfaces.fsl import RobustFOV
from nipype.interfaces.ants import N4BiasFieldCorrection, Registration
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.spm import NewSegment
from nipype.interfaces.io import SelectFiles, DataSink

In [ ]:
# Specify SPM location
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-r7219/spm12_mcr/spm12')

## Relevant Execution Variables

In [ ]:
# Folder paths and names
exp_dir = '/data/derivatives'
out_dir = 'fmriflows'
work_dir = '/workingdir'

In [ ]:
# Create fmriflows output folder if missing
import pathlib
pathlib.Path(opj(exp_dir, out_dir)).mkdir(parents=True, exist_ok=True) 

In [ ]:
# Creation of template brain with desired voxel resolution
template_dir = '/templates/mni_icbm152_nlin_asym_09c/'
brain_template = opj(template_dir, '1.0mm_brain.nii.gz')

In [ ]:
# Resample template brain to desired resolution
from nibabel import load, Nifti1Image
from nilearn.image import resample_img
from nibabel.spaces import vox2out_vox

img = load(brain_template)
target_shape, target_affine = vox2out_vox(img, voxel_sizes=res_norm)
img_resample = resample_img(img, target_affine, target_shape, clip=True)
norm_template = opj(template_dir, 'template_brain_%s.nii.gz' %'_'.join([str(n) for n in res_norm]))
img_resample.to_filename(norm_template)

## Create a subworkflow for the Main Workflow

### Implement Nodes

In [ ]:
# Reorient anatomical images to RAS
reorient = Node(Reorient(orientation='RAS'), name='reorient')

In [ ]:
# Reduces FOV of images to remove lower head and neck
cropFOV = Node(RobustFOV(output_type='NIFTI_GZ'), name='cropFOV')

In [ ]:
# Corrects bias field
n4 = Node(N4BiasFieldCorrection(dimension=3), name='n4')

In [ ]:
# Gunzips images
gunzip = Node(Gunzip(), name='gunzip')

In [ ]:
# Segments brain into 5 classes (GM, WM, CSF, Skull & Head)
segment = Node(NewSegment(), name='segment')

In [ ]:
# Compute Brain Mask and Extract Brain
def get_brain_and_mask(in_file, segments):
    
    import nibabel as nb
    from nilearn.image import clean_img, mean_img, math_img
    from scipy.ndimage.morphology import (
        binary_fill_holes, binary_dilation, binary_erosion)
    from os.path import basename, abspath

    # Load T1w corrected image
    img = nb.load(in_file)

    # Brainmask is created from the probability tissue maps
    gm, wm, csf, skull, head = [s[0] for s in segments]
    img_gmwm = math_img("(img1 + img2) >= 0.25", img1=gm, img2=wm)
    img_csf = math_img("img1 >= 1.0", img1=csf)
    img_not_rest = math_img("(img1 + img2) >= 0.25", img1=head, img2=skull)
    img_mask = math_img("(img1 + img2 - img3) >= 1.0", img1=img_gmwm, img2=img_csf, img3=img_not_rest)

    # Improves brainmask by 1 x erosion, 2 x dilation & filling of wholes
    data_mask = binary_erosion(
                binary_fill_holes(
                binary_dilation(
                img_mask.get_data(),
                    iterations = 2)),
                    iterations = 1).astype('int8')
    img_mask = nb.Nifti1Image(data_mask, img.affine, img.header)

    # Extract Brain with Mask
    img_brain = math_img("img1 * img2", img1=img, img2=img_mask)

    # Store output in nifti files
    filename = abspath(basename(in_file))
    out_file = filename.replace('.nii', '_brain.nii')
    mask = filename.replace('.nii', '_brainmask.nii')
    img_brain.to_filename(out_file)
    img_mask.to_filename(mask)

    return out_file, mask

extract_brain = Node(Function(input_names=['in_file', 'segments'],
                              output_names=['out_file', 'mask'],
                              function=get_brain_and_mask),
                     name='extract_brain')

In [ ]:
# Compress segmentation files
def compress_segments(segments):

    import nibabel as nb
    from os.path import basename, abspath
    
    # Change the compression level of the NIfTI image
    nb.openers.Opener.default_compresslevel = 6

    # Go through the individual segments
    compressed_segments = []
    for s in segments:
        new_fname  = s[0] + '.gz'
        nb.load(s[0]).to_filename(new_fname)
        compressed_segments.append(new_fname)
    
    return compressed_segments

compressor = Node(Function(input_names=['segments'],
                           output_names=['compressed_segments'],
                           function=compress_segments),
                  name='compressor')

In [ ]:
# Normalize anatomy to ICBM template
antsreg = Node(Registration(fixed_image=norm_template,
                            num_threads=1,
                            output_inverse_warped_image=True,
                            output_warped_image=True,

                            collapse_output_transforms=True,
                            dimension=3,
                            float=True,
                            initial_moving_transform_com=True,
                            interpolation='Linear',
                            transforms=['Rigid', 'Affine', 'SyN'][:n_stages],
                            transform_parameters=[(0.1,), (0.08,),
                                                  (0.1, 3.0, 0.0)][:n_stages],

                            metric=['Mattes', 'Mattes', 'CC'][:n_stages],
                            metric_weight=[1.0] * n_stages,
                            radius_or_number_of_bins=[64, 64, 4][:n_stages],
                            sampling_strategy=['Regular', 'Regular', 'None'][:n_stages],
                            sampling_percentage=[0.25, 0.25, 1][:n_stages],
                            number_of_iterations=[[1000, 500, 250, 100],
                                                  [1000, 500, 250, 100],
                                                  [100, 75, 50, 25]][:n_stages],
                            convergence_threshold=[1e-06] * n_stages,
                            convergence_window_size=[20, 20, 10][:n_stages],
                            smoothing_sigmas=[[3, 2, 1, 0]] * n_stages,
                            sigma_units=['vox'] * n_stages,
                            shrink_factors=[[8, 4, 2, 1]] * n_stages,
                            use_estimate_learning_rate_once=[True ,True, True][:n_stages],
                            use_histogram_matching=True,

                            winsorize_lower_quantile=0.005,
                            winsorize_upper_quantile=0.995,
                            write_composite_transform=True,
                            verbose=True,
                            terminal_output='file'),
               name='antsreg')

### Create Main Workflow

In [ ]:
# Create main preprocessing workflow
mainflow = Workflow(name='mainflow')

In [ ]:
# Add nodes to workflow and connect them
mainflow.connect([(reorient, cropFOV, [('out_file', 'in_file')]),
                  (cropFOV, n4, [('out_roi', 'input_image')]),
                  (n4, gunzip, [('output_image', 'in_file')]),
                  (gunzip, segment, [('out_file', 'channel_files')]),
                  (segment, extract_brain, [('native_class_images', 'segments')]),
                  (segment, compressor, [('native_class_images', 'segments')]),
                  (n4, extract_brain, [('output_image', 'in_file')]),
                  (extract_brain, antsreg, [('out_file', 'moving_image')])
                  ])

## Create a subworkflow for the report Workflow

### Implement Nodes

In [ ]:
# Create visual figures for anatomical preprocessing
def plot_figures(sub, sess, n4, segments, brain, T1_template, warped_file):
    
    import nibabel as nb
    from nilearn.plotting import plot_stat_map, plot_roi
    from matplotlib.pyplot import figure

    import numpy as np
    from nilearn.image import coord_transform
    from os.path import basename, abspath
    
    # Support Function to get optimal cut for visualization
    def get_cut_ids(img, axis=0):

        # Compute voxel id to cut
        idx = np.sort(img.get_data().nonzero()[axis])
        vox_id = np.linspace(idx.min(), idx.max(), num=12, endpoint=True).astype('int')
        vox_id = vox_id[2:-2]

        # Translate voxel id to image space
        if axis == 0:
            cut_ids = [int(coord_transform(r, 0, 0, img.affine)[0]) for r in vox_id]
        elif axis == 1:
            cut_ids = [int(coord_transform(0, r, 0, img.affine)[1]) for r in vox_id]
        elif axis == 2:
            cut_ids = [int(coord_transform(0, 0, r, img.affine)[2]) for r in vox_id]
        return cut_ids
    
    title_txt = 'sub: %s' % sub
    
    # Add session suffix if present
    if sess:
        title_txt += ' - sess: %s' % sess
    
    # Visualize Tissue Segmentation of T1w
    img = nb.load(brain)
    data = np.stack((np.zeros(img.get_data().shape),
                     nb.load(segments[0][0]).get_data(),
                     nb.load(segments[1][0]).get_data(),
                     nb.load(segments[2][0]).get_data(),
                     nb.load(segments[3][0]).get_data(),
                     nb.load(segments[4][0]).get_data()), axis= -1)
    label_id = np.argmax(data, axis=-1)
    segmentation = nb.Nifti1Image(label_id, img.affine, img.header)

    fig = figure(figsize=(16, 8))
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)

        plot_roi(segmentation, cmap='Accent', dim=1, annotate=False, bg_img=n4,
                 display_mode=e, title=title_txt + ' - %s-axis' % e,
                 resampling_interpolation='nearest',
                 cut_coords=get_cut_ids(img, i), axes=ax)
    
    out_segmentation = basename(brain).replace('brain.nii.gz', 'segmentation.svg')
    fig.savefig(out_segmentation, bbox_inches='tight', facecolor='black',
                frameon=True, dpi=300, transparent=True)

    # Visualize Brain Extraction of T1w
    fig = figure(figsize=(16, 8))
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)
        plot_stat_map(brain, title=title_txt + ' - %s-axis' % e, colorbar=False,
                      threshold='auto', bg_img=n4, cmap='magma', display_mode=e,
                      resampling_interpolation='nearest', dim=-1,
                      cut_coords=get_cut_ids(nb.load(brain), i), annotate=False, axes=ax)

    out_brain = basename(brain).replace('.nii.gz', '.svg')
    fig.savefig(out_brain, bbox_inches='tight', facecolor='black', frameon=True,
                dpi=300, transparent=True)
    
    # Visualize T1w to MNI registration
    fig = figure(figsize=(16, 8))
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)
        plot_stat_map(warped_file, title=title_txt + ' - %s-axis' % e, colorbar=False,
                      threshold='auto', bg_img=T1_template, display_mode=e,
                      resampling_interpolation='nearest',
                      cut_coords=get_cut_ids(nb.load(warped_file), i),
                      cmap='magma', annotate=False, axes=ax)
    
    out_warp = basename(warped_file).replace('.nii.gz', '.svg')
    fig.savefig(out_warp, bbox_inches='tight', facecolor='black', frameon=True,
                dpi=300, transparent=True)
    
    return abspath(out_segmentation), abspath(out_brain), abspath(out_warp), sub, sess
    
# Create Plotting Node
create_figures = Node(Function(input_names=['sub', 'sess', 'n4', 'segments', 'brain',
                                           'T1_template', 'warped_file'],
                              output_names=['out_segmentation', 'out_brain', 'out_warp',
                                            'sub', 'sess'],
                              function=plot_figures),
                name='create_figures')
create_figures.inputs.T1_template = brain_template.replace('brain', 'T1')

In [ ]:
# Write the HTML report
def write_report(sub, sess):
    
    import os
    
    with open('/templates/report_template.html', 'r') as report:
        txt = report.read()
        txt = txt.replace('sub-placeholder', 'sub-%s' % sub)
        
        # Add session suffix if present
        if sess:
            txt = txt.replace('ses-placeholder', 'ses-%s' % sess)
            filename = 'sub-%s_ses-%s.html' % (sub, sess)
        else:
            txt = txt.replace('ses-placeholder', '')
            txt = txt.replace('__', '_')
            filename = 'sub-%s.html' % sub

    report_file = os.path.join('/data', 'derivatives', 'fmriflows', filename)
    
    with open(report_file, 'w') as report:
        report.writelines(txt)

# Create Report Node
create_report = Node(Function(input_names=['sub', 'sess'],
                              function=write_report),
                     name='create_report')

### Create report Workflow

In [ ]:
# Create report workflow
reportflow = Workflow(name='reportflow')

In [ ]:
# Add nodes to workflow and connect them
reportflow.connect([(create_figures, create_report, [('sub', 'sub'),
                                                     ('sess', 'sess')
                                                     ])
                    ])

## Specify Input & Output Stream

In [ ]:
# Get all anatomical files
from bids.layout import BIDSLayout
layout = BIDSLayout('/data/')

In [ ]:
# Iterate over subject and session id
infosource = Node(IdentityInterface(fields=['subject_id', 'session_id']),
                  name='infosource')
if session_list:
    infosource.iterables = [('subject_id', subject_list),
                            ('session_id', session_list)]
else:
    infosource.iterables = [('subject_id', subject_list)]
    infosource.inputs.session_id = ''

In [ ]:
# Compute Brain Mask and Extract Brain
def create_file_path(subject_id, session_id, layout, T1w_id):

    from os.path import join
    
    entities = {'subject_id': subject_id,
                'T1w_id': T1w_id}
    
    # Add session id if present in dataset
    if session_id != '':
        entities['session_id'] = session_id
    
    pattern = 'sub-{subject_id}[/ses-{session_id}]/anat/'
    pattern += 'sub-{subject_id}[_ses-{session_id}]_{T1w_id}.nii.gz'

    fpath = layout.build_path(entities, path_patterns=[pattern])

    return join('/data', fpath)

selectfiles = Node(Function(input_names=['subject_id', 'session_id',
                                         'layout', 'T1w_id'],
                            output_names=['anat'],
                            function=create_file_path),
                   name='selectfiles')
selectfiles.inputs.layout = layout
selectfiles.inputs.T1w_id = T1w_id

In [ ]:
# Save relevant outputs in a datasink
datasink = Node(DataSink(base_directory=exp_dir,
                         container=out_dir),
                name='datasink')

In [ ]:
# Apply the following naming substitutions for the datasink
substitutions = [('_subject_id_%s/' % (sub),
                  'sub-{0}/sub-{0}_'.format(sub))
                 for sub in subject_list]
substitutions += [('_session_id_%s_subject_id_%s/' % (sub, sess),
                   'sub-{0}/sub-{0}__ses-{1}_'.format(sub, sess))
                  for sub in subject_list
                  for sess in session_list]
substitutions += [('_%s' % T1w_id, ''),
                  ('_ras', ''),
                  ('_ROI', ''),
                  ('_corrected', ''),
                  ('c1', 'seg_gm_'),
                  ('c2', 'seg_wm_'),
                  ('c3', 'seg_csf_'),
                  ('c4', 'seg_skull_'),
                  ('c5', 'seg_head_')
                 ]
substitutions += [('sub-{0}_sub-{0}'.format(sub), 'sub-%s' % sub)
                  for sub in subject_list]
substitutions += [('_sub-%s_ses-%s' % (sub, sess), '')
                  for sess in session_list
                  for sub in subject_list]
substitutions += [('_sub-%s.nii' % sub, '.nii')
                  for sub in subject_list]
substitutions += [('/sub-%s_ses-%s.nii' % (sub, sess),
                   '/sub-%s_ses-%s_T1w_corrected.nii' % (sub, sess))
                  for sess in session_list
                  for sub in subject_list]
substitutions += [('/sub-%s.nii' % sub,
                   '/sub-%s_T1w_corrected.nii' % sub)
                  for sub in subject_list]
datasink.inputs.substitutions = substitutions

## Create Preprocessing Workflow

In [ ]:
# Create anatomical preprocessing workflow
preproc_anat = Workflow(name='preproc_anat')
preproc_anat.base_dir = work_dir

preproc_anat.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                                 ('session_id', 'session_id')]),
                     ])

In [ ]:
# Add input and output nodes and connect them to the main workflow
preproc_anat.connect([(selectfiles, mainflow, [('anat', 'reorient.in_file')]),
                      
                       (mainflow, datasink, [
                           ('n4.output_image', 'preproc_anat.@n4'),
                           ('compressor.compressed_segments', 'preproc_anat.@segment'),
                           ('extract_brain.out_file', 'preproc_anat.@brain'),
                           ('extract_brain.mask', 'preproc_anat.@mask'),
                           ('antsreg.warped_image', 'preproc_anat.@warped_image'),
                           ('antsreg.inverse_warped_image', 'preproc_anat.@inverse_warped_image'),
                           ('antsreg.composite_transform', 'preproc_anat.@transform'),
                           ('antsreg.inverse_composite_transform', 'preproc_anat.@inverse_transform')]),
                     ])

In [ ]:
# Add input and output nodes and connect them to the report workflow
preproc_anat.connect([(infosource, reportflow, [('subject_id', 'create_figures.sub'),
                                                ('session_id', 'create_figures.sess')
                                               ]),

                      (reportflow, datasink, [
                          ('create_figures.out_segmentation', 'preproc_anat.@vis_segmentation'),
                          ('create_figures.out_brain', 'preproc_anat.@vis_brain'),
                          ('create_figures.out_warp', 'preproc_anat.@vis_warp'),
                      ]),
                     ])

In [ ]:
# Connect main workflow with report workflow
preproc_anat.connect([(mainflow, reportflow, [
                        ('n4.output_image', 'create_figures.n4'),
                        ('segment.native_class_images', 'create_figures.segments'),
                        ('extract_brain.out_file', 'create_figures.brain'),
                        ('antsreg.warped_image', 'create_figures.warped_file')
                        ]),
                     ])

## Visualize Workflow

In [ ]:
# Create preproc_anat output graph
preproc_anat.write_graph(graph2use='colored', format='svg', simple_form=True)

# Visualize the graph in the notebook
from IPython.display import SVG
SVG(filename=opj(preproc_anat.base_dir, 'preproc_anat', 'graph.svg'))

# Run Workflow

In [ ]:
# Run the workflow in sequential mode
res = preproc_anat.run('Linear')

In [ ]:
# Save workflow graph visualizations in datasink
preproc_anat.write_graph(graph2use='flat', format='svg', simple_form=True)
preproc_anat.write_graph(graph2use='colored', format='svg', simple_form=True)

from shutil import copyfile
copyfile(opj(preproc_anat.base_dir, 'preproc_anat', 'graph.svg'),
         opj(exp_dir, out_dir, 'preproc_anat', 'graph.svg'))
copyfile(opj(preproc_anat.base_dir, 'preproc_anat', 'graph_detailed.svg'),
         opj(exp_dir, out_dir, 'preproc_anat', 'graph_detailed.svg'));

In [ ]:
# Save template brain in `preproc_anat` folder
from  os.path import basename
new_path = '/data/derivatives/fmriflows/preproc_anat/%s' % basename(norm_template)

import shutil
shutil.move(norm_template, new_path)